In [2]:
# libraries
import numpy as np
import pandas as pd
import pandasql as ps

# web scrape
import requests
from bs4 import BeautifulSoup

# ColorSpace scrape
def colorspace(hex_base):
    
    # load ColorSpace url with hex_base
    url = f'https://mycolor.space/?hex=%23{hex_base}&sub=1'
    
    # scrape page
    data = requests.get(url)
    
    # select content of 'palette' classes
    soup = BeautifulSoup(data.text)
    class_palette = soup.select('div.palettes')[0]
    
    # parse hex codes
    hex_codes = class_palette.find_all('input')
    hex_codes = [h.get('value') for h in hex_codes]
    hex_codes = pd.DataFrame({'hex_no':hex_codes})
    
    # parse palette titles
    h2 = class_palette.find_all('h2')
    h2_ttls = []
    for h in h2:
        t = str(h).strip('</h2>')
        h2_ttls.append(t)
    palette_ttls = pd.DataFrame({'palette_ttl':h2_ttls})
        
    # create index to align hex_codes & palette_ttls
    hex_codes['hex_ind'] = hex_codes.index
    palette_ind_bgn = [6,6,4,4,5,3,4,3,4,4,5,3,3,4,4,3,4,4,4,4,3,4,4,5,5]
    palette_ind_end = np.cumsum(palette_ind_bgn)
    palette_ttls['palette_ind_bgn'] = palette_ind_end - palette_ind_bgn
    palette_ttls['palette_ind_end'] = palette_ind_end - 1
    
    # join hex_codes & palette_ttls
    sql = '''
    select b.palette_ttl
    , a.hex_no
    from hex_codes a
    inner join palette_ttls b
        on a.hex_ind >= b.palette_ind_bgn
        and a.hex_ind <= b.palette_ind_end
    '''
    palette_df = ps.sqldf(sql)
    
    # dictionary of palettes
    hex_grps = []
    for t in h2_ttls:
        grp = palette_df['hex_no'][palette_df['palette_ttl']==t].tolist()
        hex_grps.append(grp)
    palettes = dict(zip(h2_ttls, hex_grps))
            
    return palettes